In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
from helpfulfun import *
%load_ext autoreload
%autoreload 2


# Load the dataset

In [ ]:
# IMPUTAZIONE DEI MISSING VALUES
# FEATURE STUDY (STUDIO DELLE VARIABILI)
# DELETE OUTLIERS 
# STANDARDIZATION

In [ ]:
def pre_process_data(x_train, x_test, alpha=0):
    """
    With this preprocess we are going to:
    - impute missing values with the median
    - perform feature study and remove variables that are useless
    - impute outliers using alpha quantiles
    - standardize data
    """
    # Missing Values: 
    # prima di tutto cambia gli 0 nella variabile pri_jet_all_pt in -999 cosi da essere consideratoi outliers. Why? It represents       The scalar sum of the transverse momentum of all the jets of the events
    # poi applica la funzione missing_values...vedi riga 91
    
    n_rows, n_cols = x_train.shape #N number of rows, D number of cols
    missing_data = np.zeros(n_cols) #create a vector with len equal to the number of variables
    missing_80 = [] # list indicating which columns we should remove
    for regressor in range(n_cols):
        missing_data[regressor] = np.count_nonzero(x_train[:,feature]==-999)/n_rows # compute percentage of missing values per column
      
        if missing_data[regressor]>0.8: 
            missing_80.append(regressor) # se maggiore di .8 allora aggiungere alla lista di colonne da rimuovere
           
        elif missing_data[regressor]>0: # se minore di .8 ma comunque con missing presenti: imputare missing con mediana!
            impute_this = x_train[:,regressor] 
            median = np.median(impute_this[impute_this != -999])
            x_train[:,feature] = np.where(x_train[:,feature]==-999, median, x_train[:,regressor])
            x_test[:,feature] = np.where(x_test[:,feature]==-999, median, x_train[:,regreessor])
                    
    X[:,missing_80]=0
    X_test[:,missing_80]=0
    
    # Feature study:
    # Other trasformation for positive features
    x_train, x_test = log_transf(x_train, x_test) #fa una trasformazione logaritmica
    # Delete useless features
    x_train = np.delete(x_train, [15,16,18,20], 1) # rimuove features inutili.. come lo decide quali sono inutili? Forse ci sono maggiori dettagli in exploration file 
    x_test = np.delete(x_test, [15,16,18,20], 1)
    
    # Delete outliers
    x_train = outliers(x_train, alpha) # rimuove gli outliers giudicando i percentili
    x_test = outliers(x_test, alpha)
    
    # Standardization
    x_train, mean_x_train, std_x_train = standardize(x_train) # standardizza i dati.. come abiam fatto noi
    x_test, _, _ = standardize(x_test, mean_x_train, std_x_train)
    
    
     
    return x_train, x_test


### Import the dataset

In [ ]:
#### import the datasets

train_list = np.genfromtxt("train.csv", dtype=None, delimiter=",", skip_header =0, unpack=True, encoding=None)
train = np.array(train_list)

test_list = np.genfromtxt("test.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
test = np.array(test_list)

In [ ]:
x_te = test[2:].T

In [ ]:
y = train[1, 1:]
x_tr = train[2:,1: ].T.astype('float')
print("x: ", x_tr.shape, " y: ", y.shape)

In [ ]:
y_tr = np.where(y == "s",1,-1)
print(y_tr.shape, " and y[:5]: ", y_tr[:5])

In [ ]:
x_tr[:,22].astype(int)

### Pre processing

In [ ]:
for i in range(x_tr.shape[1] ):
    print(i, np.count_nonzero(abs(x_tr[:,i]) == 999.0), train[i+2, 0])


In [ ]:
#keep all except for those with many missng
x_tr_2 = np.array(np.delete(x_tr,(4,5,6,12,26,27,28), axis = 1))
train = np.array(train_list)
train = np.delete(train, (6,7,8,14,28,29,31), axis= 0)
for i in range(x_tr_2.shape[1] ):
    print(i, np.count_nonzero(abs(x_tr_2[:,i]) == 999.0), train[i+2,0])


In [ ]:
#impute using median
x_tr_2[abs(x_tr_2)==999.0] = np.nan
ind = np.where(np.isnan(x_tr_2))

median = np.nanmedian(x_tr_2, axis=0, out=None, overwrite_input=False)

x_tr_2[ind] =  np.take(median, ind[1])

x_tr_2.shape


In [ ]:
np.isnan(x_tr_2)

In [ ]:
#create x_te_2
x_te_2 = np.array(np.delete(x_te,(4,5,6,12,26,27,28), axis = 1))


In [ ]:
## export the dataset:
np.savetxt("x_tr_2.csv", x_tr_2, fmt="%s", delimiter=",")
np.savetxt("x_te_2.csv", x_te_2, fmt="%s", delimiter=",")

In [ ]:
# let's remove the primitive variables:
x_tr_3 = x_tr_2[:,:9]
x_tr_3.shape
for i in range(x_tr_3.shape[1] ):
    print(i, np.count_nonzero(abs(x_tr_3[:,i]) == 999.0), train[i+2,0])


In [ ]:
# now impute outliers

def impute_outliers_mean(a):
    '''impute outliers with mean'''
    # 1. Determine mean and standard deviation
    mean = np.mean(a, axis = 0)
    std_dev = np.std(a, axis = 0)
    # 2. Normalize array around 0
    zero_based = abs(a - mean)
    # 3. Define maximum number of standard deviations
    max_deviations = 2
    # 4. Access only non-outliers using Boolean Indexing
    num_out = 0
    for i in range(a.shape[1]):
        for j in range(a.shape[0]):
            if zero_based[j,i] > max_deviations * std_dev[i]:
                a[j,i] = mean[i]
                num_out += 1
    print(num_out, "observations have been replaced by the corresponding mean")
    
    return a

#x_tr_3_no_out = impute_outliers_mean(x_tr_3)



In [ ]:
x_te_3 = x_te_2[:,:9] 

In [ ]:
## export the dataset:
np.savetxt("x_tr_3_no_out.csv", x_tr_3_no_out, fmt="%s", delimiter=",")
np.savetxt("x_te_3_no_out.csv", x_te_3, fmt="%s", delimiter=",")

In [ ]:
x_tr_4 = np.array(np.delete(x_tr,(0,5,6,9,21,23,29), axis = 1))
x_te_4 = np.array(np.delete(x_te,(0,5,6,9,21,23,29), axis = 1))
train = np.array(train_list)
train = np.delete(train, (2,7,8,11,23,25,31), axis = 0)

In [ ]:
for i in range(x_tr_4.shape[1] ):
    print(i, np.count_nonzero(abs(x_tr[:,i]) == 999.0), train[i+2, 0])


In [ ]:
#impute nan using median
x_tr_4[abs(x_tr_4)==999.0] = np.nan
ind = np.where(np.isnan(x_tr_4))

median = np.nanmedian(x_tr_4, axis=0, out=None, overwrite_input=False)

x_tr_4[ind] =  np.take(median, ind[1])

x_tr_4

In [ ]:
x_tr_4_no_out = impute_outliers_mean(x_tr_4)

In [ ]:
np.savetxt("x_tr_4_no_out.csv", x_tr_4_no_out, fmt="%s", delimiter=",")
np.savetxt("x_te_4_no_out.csv", x_te_4, fmt="%s", delimiter=",")